#  Identifying Bad Loans

#### Objective
To identify loans at high risk of becoming non-performing — specifically those that have reached the 2nd delinquency period (i.e. Late for 31 - 120 days), Default, or Charged Off — by analyzing borrower characteristics.

#### Metrics
Providing a loan to a defaulter results in direct financial loss. Assuming that lenders are risk averse and are more willing to miss a good borrower than fund a bad one. Hence, I seek to minimise the number of defaulters incorrectly classified as good. 

#### Goal
To construct a model that achieves a recall of ≥70%. F1 score should be at least 60% so that a balance of precision and recall is achieved.

## Importing libraries

In [ ]:
# Import libraries for Exploratory Data Analysis
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

import numpy as np
np.set_printoptions(threshold=np.inf)

# Import libraries for visualisation
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Importing the data set

In [ ]:
loan_df = pd.read_csv('./Credit Risk_Lending Club/Data_Lending Club_Jun 2007_to_Sep 2020.csv',
                                low_memory=False)

print(loan_df.shape)
loan_df.head()

In [ ]:
data_dictionary = pd.read_csv('./Credit Risk_Lending Club/Data_Dictionary_Lending Club_Jun 2007_to_Sep 2020.csv',
                          usecols=['Column','Description'],index_col=0)
data_dictionary.head()

## Building Functions

#### Defining a Column

In [ ]:
def define_col(col_name):
    print(data_dictionary.loc[col_name]['Description'])

#### Selecting a Feature

In [ ]:
def cont_stat_sig_test(df,col):
    # defining function for normality + statistical significance check
    # Step 1: normality test
    stat, p_value = stats.normaltest(df[col].values)
    
    # Step 2: filter series    
    good_loan = loan_individual_filled_df[loan_individual_filled_df['updated_loan_status']=='Good Loan'][col].values
    bad_loan = loan_individual_filled_df[loan_individual_filled_df['updated_loan_status']=='Bad Loan'][col].values
    
    # Step 3: determining appropriate statistical significance test based on normality
    if p_value>0.01:
        distribution = 'normal distribution'
        test = 'Anova test'
        stat, p_value = stats.f_oneway(good_loan, bad_loan)
    
    else:
        distribution = 'non-normal distribution'
        test = 'KS test'
        stat, p_value = stats.ks_2samp(good_loan, bad_loan)
    
    # Step 4: conclude on statistical significance
    if p_value>0.05:
        sig = 'not statistically significant'
    else:
        sig = 'statistically significant'

    print(f'{col} has {distribution}: running {test}. result: {sig}')
    return p_value

## Creating the SQLite database

In [ ]:
# Import SQLite3 to extract the data set
import sqlite3

In [ ]:
# Creating a new SQlite database
con = sqlite3.connect('loan_df.db')

In [ ]:
# Load data set to SQLite database the df in the test_db db
loan_df.to_sql(name='loan_df',con=con, index=False, if_exists='replace')

In [ ]:
# Query the first five rows of all columns
sql_query_first_5_row = '''
SELECT * 
FROM loan_df
LIMIT 5
'''

sql_query_first_5_row = pd.read_sql(sql_query_first_5_row,con)

sql_query_first_5_row

## Feature Engineering and Preliminary EDA

In [ ]:
# Displaying the columns, the number of null values, and data type

loan_df.info(verbose=True, show_counts=True)

### Scoping the Data Set to Individual Loans (i.e. remove non-individual loan applications)

In [ ]:
loan_df['application_type'].unique()

In [ ]:
loan_individual_df = loan_df[loan_df['application_type'] == 'Individual'].reset_index(drop=True)

### Identifying % of NaN in each Column and sorting them in descending order

In [ ]:
# checking columns with missing values
missing_values_df = pd.DataFrame(loan_individual_df.isna().sum()).reset_index()
missing_values_df.columns = ['col', 'num_missing']
missing_values_df['pct_missing'] = missing_values_df['num_missing']*100/len(loan_df)
missing_values_df[missing_values_df['num_missing']>0].round(3)

missing_values_descending = (
    missing_values_df[missing_values_df['num_missing'] > 0]
    .sort_values('pct_missing', ascending=False)
    .round(5)
)

# Count the number of such columns
num_cols_with_missing = missing_values_descending.shape[0]
print(f"Number of columns with missing values: {num_cols_with_missing}")

missing_values_descending

### Dropping columns that are >10% NaN

In [ ]:
# Identify columns with >10% missing
NaN_threshold = 10.0
missing_10 = missing_values_descending[missing_values_descending['pct_missing'] > NaN_threshold]

cols_to_drop = missing_10['col'].tolist()
no_cols_to_drop = len(cols_to_drop)
print(f"Number of columns with at least 10% NaN: {no_cols_to_drop}")

loan_individual_90_df = loan_individual_df.drop(columns=cols_to_drop)

missing_10

#### Dropping Columns irrelevant to Analysis

In [ ]:
irrelevant_cols = ['Unnamed: 0','id', 
                   'funded_amnt', 'funded_amnt_inv',  # alternatives to 'loan_amnt'
                   'emp_title', #not standardised titles
                   'pymnt_plan', # contains only 1 value i.e. 'n'
                   'url', # url of lending_club website
                   'zip_code', # masked value
                   'title', # alternative to purpose
                   'policy_code', # contains only 1 value i.e. '1'
                   'initial_list_status', #initial listing status (whole vs. fractional).There are do not appear to be any meaningful differences between whole and fractional loans.
                   'total_rev_hi_lim', # 'na' in data_dictionary
                   'application_type' # data set scoped to 'application_type' == 'Individual'
                  ]

loan_individual_90_df = loan_individual_90_df.drop(irrelevant_cols, axis=1)

### Dropping all rows with NaN values for remaining columns

# RESTART HERE FOR loan_individual_filled_df

In [ ]:
filled_90 = missing_values_descending[missing_values_descending['pct_missing'] < NaN_threshold]

cols_to_drop = filled_90['col'].tolist()
no_cols_to_drop = len(cols_to_drop)
print(f"Number of columns with at least 1 row with NaN: {no_cols_to_drop}")

# Drop all rows with NaN values
loan_individual_filled_df = loan_individual_90_df.dropna()

filled_90

### Print the shape before and after dropping columns >10% and rows with NaNs for comparison

In [ ]:
print(f"Original shape of data set: {loan_df.shape}")
print(f"Shape of data set with columns of individual loans: {loan_individual_df.shape}")
print(f"Shape of data set with columns of individual loans with >90% filled rows: {loan_individual_90_df.shape}")
print(f"Shape of data set of individual loans with columns fully filled: {loan_individual_filled_df.shape}")

In [ ]:
loan_individual_filled_df = loan_individual_filled_df.copy()

In [ ]:
loan_individual_filled_df.info()

In [ ]:
loan_individual_filled_df.head()

### Formatting the Columns

#### Converting to DateTime 

In [ ]:
loan_individual_filled_df["issue_d"] = pd.to_datetime(loan_individual_filled_df["issue_d"], format="%b-%Y", errors='coerce')
loan_individual_filled_df["earliest_cr_line"] = pd.to_datetime(loan_individual_filled_df["earliest_cr_line"], format="%b-%Y", errors='coerce')
loan_individual_filled_df["last_pymnt_d"] = pd.to_datetime(loan_individual_filled_df["last_pymnt_d"], format="%b-%Y", errors='coerce')
loan_individual_filled_df["last_credit_pull_d"] = pd.to_datetime(loan_individual_filled_df["last_credit_pull_d"], format="%b-%Y", errors='coerce')

print("Data type of 'issue_d' after conversion:", loan_individual_filled_df['issue_d'].dtype)
print("Data type of 'earliest_cr_line' after conversion:", loan_individual_filled_df['earliest_cr_line'].dtype)
print("Data type of 'last_pymnt_d' after conversion:", loan_individual_filled_df['last_pymnt_d'].dtype)
print("Data type of 'last_credit_pull_d' after conversion:", loan_individual_filled_df['last_credit_pull_d'].dtype)

#### Converting to Float or Integers

In [ ]:
# Convert 'int_rate', 'bc_util', 'revol_util' to float
print("Data type of 'term' before conversion:", loan_individual_filled_df['term'].dtype)
print("Data type of 'int_rate' before conversion:", loan_individual_filled_df['int_rate'].dtype)
print("Data type of 'revol_util' before conversion:", loan_individual_filled_df['revol_util'].dtype)


# If the interest rate is stored as a string with '%' (e.g., "13.56%"), 
# remove the '%' and convert the result into a float
loan_individual_filled_df['term'] = loan_individual_filled_df['term'].str.extract(r'(\d+)').astype(int)
loan_individual_filled_df['int_rate'] = loan_individual_filled_df['int_rate'].str.rstrip('%').astype(float)
loan_individual_filled_df['revol_util'] = loan_individual_filled_df['revol_util'].str.rstrip('%').astype(float)


# Verify the data type after conversion
print("Data type of 'term' after conversion:", loan_individual_filled_df['term'].dtype)
print("Data type of 'int_rate' after conversion:", loan_individual_filled_df['int_rate'].dtype)
print("Data type of 'revol_util' after conversion:", loan_individual_filled_df['revol_util'].dtype)

### Feature Engineering

#### Function to convert 'emp_length' to 'emp_length_int'

In [ ]:
def convert_emp_length(emp_str):
    emp_str = emp_str.strip()
    # Recode "< 1 year" as 0
    if emp_str == "< 1 year":
        return 0
    # Recode "10+ years" as 10
    if emp_str == "10+ years":
        return 10
    # Otherwise, extract the numeric portion (e.g., "3 years" -> 3)
    try:
        return int(emp_str.split()[0])
    except Exception:
        return np.nan

In [ ]:
# Apply the conversion function to the 'emp_length' column
loan_individual_filled_df['emp_length_int'] = loan_individual_filled_df['emp_length'].apply(convert_emp_length)

loan_individual_filled_df.drop(['emp_length'], axis=1,inplace=True) # to drop 'emp_length' after conversion

#### Function to assign the category of 'Bad Loan' and 'Good Loan' to loan_status

In [ ]:
def categorise_status(status):
    status = str(status).strip()
    if status == "Charged Off":
        return "Bad Loan"
    elif status == "Does not meet the credit policy. Status:Charged Off":
        return "Bad Loan"
    elif status == "Default":
        return "Bad Loan"
    elif status == "Late (31-120 days)":
        return "Bad Loan"
    else:
        return "Good Loan"

In [ ]:
# Apply the conversion function to the 'loan_status' column
loan_individual_filled_df['updated_loan_status'] = loan_individual_filled_df['loan_status'].apply(categorise_status)

loan_individual_filled_df.drop(['loan_status'], axis=1,inplace=True) # to drop 'loan_status' after conversion

#### Target Variable (i.e. Bad Loans) as '1'

In [ ]:
loan_individual_filled_df['bad_loan'] = loan_individual_filled_df['updated_loan_status'].apply(lambda x: 0 if x == "Good Loan" else 1)

#### 'verification_status' to 'cleaned_verification_status'

In [ ]:
loan_individual_filled_df['verification_status'].unique()

In [ ]:
# Define 'verified' values
verified_values = ['Verified', 'Source Verified']

# Create a new 'cleaned_verification' column
loan_individual_filled_df['cleaned_verification_status'] = np.where(
    loan_individual_filled_df['verification_status'].isin(verified_values),
    'Verified',
    'Not Verified'
)

loan_individual_filled_df.drop(['verification_status'], axis=1,inplace=True) # to drop 'verification_status' after conversion

#### 'region' of the US

In [ ]:
# Define lists of states by region
regions = {
    'Northeast': ['CT','ME','MA','NH','RI','VT','NJ','NY','PA'],
    'Midwest':   ['IN','IL','MI','OH','WI','IA','KS','MN','MO','NE','ND','SD'],
    'South':     ['DE','FL','GA','MD','NC','SC','VA','DC','WV','AL','KY','MS','TN','AR','LA','OK','TX'],
    'West':      ['AZ','CO','ID','MT','NV','NM','UT','WY','AK','CA','HI','OR','WA']
}

# Mapping US states to their regions
state_to_region = {
    state: region
    for region, states in regions.items()
    for state in states
}

# Apply the mapping
loan_individual_filled_df['region'] = loan_individual_filled_df['addr_state'].map(state_to_region)

#### 'avg_FICO'

In [ ]:
loan_individual_filled_df['avg_FICO'] = (loan_individual_filled_df['fico_range_low'] + loan_individual_filled_df['fico_range_high'])/2

# to drop 'fico_range_low' and 'fico_range_high' after conversion
loan_individual_filled_df.drop(['fico_range_low', 'fico_range_high'], axis=1,inplace=True)

#### 'avg_last_FICO'

In [ ]:
loan_individual_filled_df['avg_last_FICO'] = (loan_individual_filled_df['last_fico_range_low'] + loan_individual_filled_df['last_fico_range_high'])/2

# to drop 'last_fico_range_low' and 'last_fico_range_high' after conversion
loan_individual_filled_df.drop(['last_fico_range_low', 'last_fico_range_high'], axis=1,inplace=True)

#### 'median_household_income_2019' and 'rank_of_state_by_median_household_income_2019'

In [ ]:
loan_individual_filled_df["addr_state"].unique()

In [ ]:
loan_individual_filled_df["addr_state"].nunique()

In [ ]:
# 1. Load the ACS S1901 table (2019 1‑year estimates)
acs = pd.read_csv('./Credit Risk_Lending Club/ACSST1Y2019.S1901-2025-04-20T165846.csv',dtype=str)

# 2. Isolate the row containing the median household income estimate
median_row = acs.loc[acs['Label (Grouping)'] == 'Median income (dollars)']

# 3. Transpose so each state becomes a row
med = (
    median_row
    .set_index('Label (Grouping)')
    .T
    .reset_index()
    .rename(columns={'index': 'state_field', 0: 'Median_Household_Income'})
)

# 4. split state_field into state / category / measure
med[['state','category','measure']] = (
    med['state_field']
       .str.split('!!', expand=True)
)

# 5. Clean and convert income to numeric
med['Median_Household_Income'] = med['Median income (dollars)'].str.replace(',', '').astype(int)

# 6. Keep only the rows where 'category' == 'Households'
households_income = med[(med['category'].str.lower() == 'households')].copy()

# 7. Map full state names to USPS abbreviations
us_state_abbrev = {
    'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR',
    'California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE',
    'District of Columbia': 'DC','Florida': 'FL','Georgia': 'GA','Hawaii': 'HI',
    'Idaho': 'ID','Illinois': 'IL','Indiana': 'IN','Iowa': 'IA',
    'Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME',
    'Maryland': 'MD','Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN',
    'Mississippi': 'MS','Missouri': 'MO','Montana': 'MT','Nebraska': 'NE',
    'Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM',
    'New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH',
    'Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI',
    'South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX',
    'Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA',
    'West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY','Puerto Rico': 'PR'
}
households_income['state_abbrev'] = households_income['state'].map(us_state_abbrev)

# 8. Build a mapping dictionary
income_map = dict(zip(households_income['state_abbrev'], households_income['Median_Household_Income']))

# 9. Map the median income into your loans DataFrame
loan_individual_filled_df['median_household_income_2019'] = loan_individual_filled_df['addr_state'].map(income_map)

# 10. Rank in descending order (highest income = rank 1)
loan_individual_filled_df['rank_of_state_by_median_household_income_2019'
] = (
    loan_individual_filled_df['median_household_income_2019'
].rank(ascending=False, method='dense').astype(int)
)

# 11. Print any state that does not have a rank
missing = loan_individual_filled_df.loc[
loan_individual_filled_df["rank_of_state_by_median_household_income_2019"].isna(), "addr_state"].unique()
if len(missing):
    print("Warning—these addr_state codes weren’t found in the median‑income map:", missing)
    
# View the first few rows to verify
loan_individual_filled_df[['addr_state', 'median_household_income_2019','rank_of_state_by_median_household_income_2019']].head()

#### 'annual_income_more_than_median_household_income_2019'

In [ ]:
loan_individual_filled_df['annual_income_more_than_median_household_income_2019'] = (
    loan_individual_filled_df['annual_inc'] > loan_individual_filled_df['median_household_income_2019']
).map({True: 'yes', False: 'no'})

#### '%_more_or_less'

In [ ]:
loan_individual_filled_df['%_more_or_less_than_median'] = (
    (loan_individual_filled_df['annual_inc'] - loan_individual_filled_df['median_household_income_2019']) 
    / loan_individual_filled_df['median_household_income_2019'] 
    * 100
).round(2)

In [ ]:
# Verify new columns
loan_individual_filled_df[['annual_inc', 'median_household_income_2019',
    'annual_income_more_than_median_household_income_2019', '%_more_or_less_than_median']].head()

#### 'number_of_months_bet_issued_and_earliest_cr_line'

In [ ]:
loan_individual_filled_df['number_of_months_bet_issued_and_earliest_cr_line'] = (
    (loan_individual_filled_df['issue_d'].dt.year - loan_individual_filled_df['earliest_cr_line'].dt.year) * 12
  + (loan_individual_filled_df['issue_d'].dt.month - loan_individual_filled_df['earliest_cr_line'].dt.month)
)

# 4) (Optional) Drop the temp datetime columns
# df.drop(columns=['issue_date','earliest_cr_line_date'], inplace=True)

loan_individual_filled_df[['issue_d','earliest_cr_line','number_of_months_bet_issued_and_earliest_cr_line']].head()

In [ ]:
cleaned_loan_individual_filled_df = loan_individual_filled_df.copy()

### Preliminary Exploratory Data Analysis

#### Breakdown of 'updated_loan_status'

In [ ]:
# Calculate normalized distribution (percentages) for the loan status column.
updated_loan_status_breakdown = cleaned_loan_individual_filled_df['updated_loan_status'].value_counts(normalize=True).reset_index()
updated_loan_status_breakdown.columns = ['updated_loan_status', 'percentage']
updated_loan_status_breakdown['percentage'] *= 100  # Convert to actual percentage

# Create an interactive bar chart
updated_loan_status_fig = px.bar(
    updated_loan_status_breakdown,
    x='updated_loan_status',
    y='percentage',
    labels={'loan_status_breakdown': 'Updated_Loan Status', 'percentage': 'Percentage'},
    title='Breakdown of updated_loan_status Column',
    text='percentage'
)

# Customize text display and rotate x-axis labels for better readability.
updated_loan_status_fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside', cliponaxis=False)
updated_loan_status_fig.update_layout(xaxis_tickangle=0)

# Display the interactive plot
updated_loan_status_fig.show()

#### 'updated_loan_status' by 'loan_amnt'

In [ ]:
# Create a box plot of loan amount by updated loan status
fig = px.box(
    cleaned_loan_individual_filled_df,
    x="updated_loan_status",
    y="loan_amnt",
    title="Distribution of Loan Amounts by Updated Loan Status",
    labels={
        "updated_loan_status": "Updated Loan Status",
        "loan_amnt": "Loan Amount (USD)"
    }
)

fig.show()

#### Correlation Matrix

In [ ]:
# Set figure size
plt.figure(figsize=(18, 12))  # Increased size for better readability

# Generate the heatmap with improved settings
sns.heatmap(cleaned_loan_individual_filled_df.corr(numeric_only=True),
            annot=True,               # Display values on the heatmap
            cmap='viridis',           # Color map
            annot_kws={"size": 10},   # Adjust annotation size
            fmt='.2f',                # Format annotation to 2 decimal places
            linewidths=0.5,           # Add some spacing between the cells
            cbar_kws={'shrink': 0.8}, # Reduce the colorbar size
            xticklabels=cleaned_loan_individual_filled_df.corr(numeric_only=True).columns,  # Ensure correct column labels
            yticklabels=cleaned_loan_individual_filled_df.corr(numeric_only=True).columns   # Ensure correct row labels
            )

# Rotate x and y axis labels for better visibility
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(rotation=0, ha='right', fontsize=12)

# Title for the heatmap
plt.title('Correlation of Features', fontsize=16)

# Show the plot
plt.tight_layout()  # Adjust layout to ensure no clipping
plt.show()

In [ ]:
# 1. Compute the absolute correlation matrix
corr_matrix = cleaned_loan_individual_filled_df.corr(numeric_only=True).abs()

# 2. Build a mask to exclude self‑correlations (the diagonal)
mask = np.eye(corr_matrix.shape[0], dtype=bool)

# 3. Unstack the off‑diagonal entries into a long table
corr_pairs = (
    corr_matrix
      .where(~mask)      # drop diagonal
      .stack()           # keep only non‑NaN entries
      .reset_index()     # to DataFrame
)
corr_pairs.columns = ['Feature 1', 'Feature 2', 'Correlation']

# 4. Filter for high correlations (e.g. > 0.6)
threshold = 0.6
high_corr = corr_pairs[corr_pairs['Correlation'] > threshold].copy()

# 5. Add original column index for each feature
high_corr['idx1'] = high_corr['Feature 1'].map(lambda f: loan_individual_filled_df.columns.get_loc(f))
high_corr['idx2'] = high_corr['Feature 2'].map(lambda f: loan_individual_filled_df.columns.get_loc(f))

# 6. Remove duplicate pairs (so (A,B) & (B,A) appear only once)
high_corr['Ordered_Pair'] = high_corr.apply(
    lambda r: tuple(sorted((r['Feature 1'], r['Feature 2']))),
    axis=1
)
high_corr = (
    high_corr
      .drop_duplicates('Ordered_Pair')
      .drop(columns='Ordered_Pair')
      .sort_values('Correlation', ascending=False)
      .reset_index(drop=True)
)

# 7. Reorder columns for clarity
high_corr = high_corr[['idx1','Feature 1','idx2','Feature 2','Correlation']]

# 8. Print the result
print("Highly correlated feature pairs (|corr| > 0.6) with original indices:")
print(high_corr.to_string(index=False))

#### Distribution of 'loan_amt'

In [ ]:
# Plot interactive histogram of loan amounts
loan_amt_fig = px.histogram(
    cleaned_loan_individual_filled_df,
    x="loan_amnt",
    nbins=50,
    title="Distribution of loan_amt",
    labels={"loan_amnt": "Loan Amount", "count": "Number of Loans"}
)

loan_amt_fig.update_layout(
    xaxis_title="Loan Amount",
    yaxis_title="Number of Loans",
    margin=dict(t=80, b=50)
)

loan_amt_fig.show()

#### Distribution of 'loan_amt' by 'updated_loan_status'

In [ ]:
good_df = cleaned_loan_individual_filled_df[cleaned_loan_individual_filled_df["updated_loan_status"] == "Good Loan"]
bad_df  = cleaned_loan_individual_filled_df[cleaned_loan_individual_filled_df["updated_loan_status"] == "Bad Loan"]

# Create subplots: 1 row, 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Good Loans", "Bad Loans"),
    shared_yaxes=True,
    horizontal_spacing=0.1
)

# Add histogram for Good Loans
fig.add_trace(
    go.Histogram(
        x=good_df["loan_amnt"],
        histnorm="percent",
        nbinsx=50,
        marker_color="green",
        showlegend=False
    ),
    row=1, col=1
)

# Add histogram for Bad Loans
fig.add_trace(
    go.Histogram(
        x=bad_df["loan_amnt"],
        histnorm="percent",
        nbinsx=50,
        marker_color="red",
        showlegend=False
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text="Distribution of Loan Amount by Updated Loan Status",
    barmode="overlay",
    margin=dict(t=80, b=50, l=50, r=50)
)

# Update axes labels
fig.update_xaxes(title_text="Loan Amount (USD)", row=1, col=1)
fig.update_xaxes(title_text="Loan Amount (USD)", row=1, col=2)
fig.update_yaxes(title_text="Percentage of Loans (%)", row=1, col=1)

# Display the interactive figure
fig.show()

#### Breakdown of 'updated_loan_status' by 'int_rate'

In [ ]:
# Compute count and percentage by interest rate and status
grouped = (
    cleaned_loan_individual_filled_df.groupby(["int_rate", "updated_loan_status"])
      .size()
      .reset_index(name="count")
)
grouped["total"] = grouped.groupby("int_rate")["count"].transform("sum")
grouped["percentage"] = grouped["count"] / grouped["total"] * 100

# Plot scatter of percentage by interest rate
fig = px.scatter(
    grouped,
    x="int_rate",
    y="percentage",
    color="updated_loan_status",
    title="Percentage of Good vs. Bad Loans by Interest Rate",
    labels={
        "int_rate": "Interest Rate (%)",
        "percentage": "Percentage of Loans (%)",
        "updated_loan_status": "Loan Status"
    },
    color_discrete_map={"Good Loan": "green", "Bad Loan": "red"},
    symbol="updated_loan_status",
    opacity=0.7
)

fig.update_layout(
    xaxis=dict(tickformat=".1f"),
    yaxis=dict(range=[0, 100]),
    legend_title_text="Loan Status"
)

fig.show()

#### 'int_rate' by 'grade' and 'sub_grade'

In [ ]:
# Compute average interest rate by grade and sub_grade
avg_by_grade = cleaned_loan_individual_filled_df.groupby('grade')['int_rate'].mean()
avg_by_sub_grade = cleaned_loan_individual_filled_df.groupby('sub_grade')['int_rate'].mean()

# Create subplots: 1 row, 2 columns
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Average Interest Rate by Grade", "Average Interest Rate by Sub_Grade"),
    horizontal_spacing=0.1
)

# Add line traces
fig.add_trace(go.Scatter(x=avg_by_grade.index,     y=avg_by_grade.values,     mode='lines+markers', name='Grade'),     row=1, col=1)
fig.add_trace(go.Scatter(x=avg_by_sub_grade.index, y=avg_by_sub_grade.values, mode='lines+markers', name='Sub_Grade'), row=1, col=2)

# Layout
fig.update_layout(title="Average Interest Rate by Grade & Sub-Grade", showlegend=False)
fig.update_xaxes(title_text="Grade",      row=1, col=1)
fig.update_xaxes(title_text="Sub-Grade", row=1, col=2)
fig.update_yaxes(title_text="Avg Interest Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Avg Interest Rate (%)", row=1, col=2)

fig.show()

#### 'updated_loan_status' by 'grade' and 'sub_grade'

In [ ]:
# Compute percentage of Bad Loans by grade
grp_grade = (
    cleaned_loan_individual_filled_df.groupby(['grade', 'updated_loan_status'], observed=True)
      .size()
      .reset_index(name='count')
)
grp_grade['total'] = grp_grade.groupby('grade', observed=True)['count'].transform('sum')
grp_grade['percent'] = grp_grade['count'] / grp_grade['total'] * 100
bad_pct_grade = grp_grade[grp_grade['updated_loan_status'] == 'Bad Loan']

# Compute percentage of Bad Loans by sub_grade
grp_sub = (
    cleaned_loan_individual_filled_df.groupby(['sub_grade', 'updated_loan_status'], observed=True)
      .size()
      .reset_index(name='count')
)
grp_sub['total'] = grp_sub.groupby('sub_grade', observed=True)['count'].transform('sum')
grp_sub['percent'] = grp_sub['count'] / grp_sub['total'] * 100
bad_pct_sub = grp_sub[grp_sub['updated_loan_status'] == 'Bad Loan']

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Bad Loan % by Grade", "Bad Loan % by Sub_Grade"),
    horizontal_spacing=0.12
)

# Add line traces
fig.add_trace(
    go.Scatter(
        x=bad_pct_grade['grade'],
        y=bad_pct_grade['percent'],
        mode='lines+markers',
        line=dict(width=2),
        marker=dict(size=6),
        name='Bad % by Grade'
    ),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=bad_pct_sub['sub_grade'],
        y=bad_pct_sub['percent'],
        mode='lines+markers',
        line=dict(width=2),
        marker=dict(size=4),
        name='Bad % by Sub_Grade'
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text="Percentage of Bad Loans by Grade & Sub_Grade",
    showlegend=False,
    margin=dict(t=80, b=50, l=50, r=50)
)
fig.update_xaxes(title_text="Grade", row=1, col=1)
fig.update_xaxes(title_text="Sub_Grade", row=1, col=2)
fig.update_yaxes(title_text="Percentage of Bad Loans (%)", row=1, col=1)
fig.update_yaxes(title_text="Percentage of Bad Loans (%)", row=1, col=2)

# Show the plot
fig.show()


#### Breakdown of 'loan_status' by 'purpose'

In [ ]:
# Compute counts & percentages by purpose
grp = (
    cleaned_loan_individual_filled_df.groupby(["purpose","updated_loan_status"], observed=True)
      .size()
      .reset_index(name="count")
)
grp["total"]   = grp.groupby("purpose", observed=True)["count"].transform("sum")
grp["percent"] = grp["count"] / grp["total"] * 100

# Determine x-axis order by descending Bad‐Loan %
bad_pct = grp[grp["updated_loan_status"]=="Bad Loan"][["purpose","percent"]]
purpose_order = bad_pct.sort_values("percent", ascending=False)["purpose"].tolist()

# Plot without barnorm (y="percent" directly)
fig = px.bar(
    grp,
    x="purpose",
    y="percent",
    color="updated_loan_status",
    barmode="group",
    category_orders={"purpose": purpose_order},
    color_discrete_map={"Good Loan":"green","Bad Loan":"red"},
    title="Percentage of Good vs. Bad Loans by Purpose",
    labels={
      "purpose":"Loan Purpose",
      "updated_loan_status":"Loan Status",
      "percent":"Percentage of Loans (%)"
    },
    text="percent"
)

# Format text and layout
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis=dict(title="Percentage of Loans (%)", range=[0,100]),
    legend_title_text="Loan Status",
    margin=dict(t=80, b=150)
)

fig.show()

#### Distribution of 'annual_inc'

In [ ]:
cleaned_loan_individual_filled_df['annual_inc'].describe()

In [ ]:
# Filter for annual income up to 75 percentile (i.e. less than 100,000)
df_filtered = cleaned_loan_individual_filled_df[cleaned_loan_individual_filled_df['annual_inc'] < 100000]

# Plot the distribution
fig = px.histogram(
    df_filtered,
    x='annual_inc',
    nbins=50,
    title='Distribution of Annual Income (< $100,000)',
    labels={'annual_inc': 'Annual Income (USD)', 'count': 'Number of Loans'}
)
fig.update_layout(xaxis_title='Annual Income (USD)', yaxis_title='Number of Loans')
fig.show()

In [ ]:
# Filter for annual income less than 1,000,000
df_filtered = cleaned_loan_individual_filled_df[cleaned_loan_individual_filled_df['annual_inc'] < 1000000]

# Plot the distribution
fig = px.histogram(
    df_filtered,
    x='annual_inc',
    nbins=500,
    title='Distribution of Annual Income (< $1,000,000)',
    labels={'annual_inc': 'Annual Income (USD)', 'count': 'Number of Loans'}
)
fig.update_layout(xaxis_title='Annual Income (USD)', yaxis_title='Number of Loans')
fig.show()

#### Breakdown of 'loan_status' by 'annual_inc'

In [ ]:
grouped = (
    cleaned_loan_individual_filled_df.groupby(["annual_inc", "updated_loan_status"], observed=True)
      .size()
      .reset_index(name="count")
)
grouped["total"] = grouped.groupby("annual_inc", observed=True)["count"].transform("sum")
grouped["percentage"] = grouped["count"] / grouped["total"] * 100

# Plot scatter of percentage breakdown by annual income
fig = px.scatter(
    grouped,
    x="annual_inc",
    y="percentage",
    color="updated_loan_status",
    title="Percentage of Good vs. Bad Loans by Annual Income",
    labels={
        "annual_inc": "Annual Income (USD)",
        "percentage": "Percentage of Loans (%)",
        "updated_loan_status": "Loan Status"
    },
    color_discrete_map={"Good Loan": "green", "Bad Loan": "red"},
    symbol="updated_loan_status",
    opacity=0.7
)

fig.update_layout(
    xaxis_title="Annual Income (USD)",
    yaxis=dict(title="Percentage of Loans (%)", range=[0, 100]),
    legend_title_text="Loan Status"
)

fig.show()

#### Breakdown of 'loan_status' by 'dti'

In [ ]:
# Group by dti and status to compute percentage breakdown
grouped = (
    cleaned_loan_individual_filled_df.groupby(["dti", "updated_loan_status"], observed=True)
      .size()
      .reset_index(name="count")
)
grouped["total"] = grouped.groupby("dti", observed=True)["count"].transform("sum")
grouped["percentage"] = grouped["count"] / grouped["total"] * 100

# Plot scatter of percentage breakdown by DTI
fig = px.scatter(
    grouped,
    x="dti",
    y="percentage",
    color="updated_loan_status",
    title="Percentage of Good vs. Bad Loans by Debt-to-Income Ratio (DTI)",
    labels={
        "dti": "Debt-to-Income Ratio (DTI)",
        "percentage": "Percentage of Loans (%)",
        "updated_loan_status": "Loan Status"
    },
    color_discrete_map={"Good Loan": "green", "Bad Loan": "red"},
    opacity=0.7
)

fig.update_layout(
    xaxis_title="DTI",
    yaxis_title="Percentage of Loans (%)",
    legend_title_text="Loan Status",
    yaxis=dict(range=[0, 100])
)

fig.show()

#### Breakdown of 'updated_loan_status' by 'avg_FICO'

In [ ]:
# Group by avg_FICO and status to compute percentage breakdown
grouped = (
    cleaned_loan_individual_filled_df.groupby(["avg_FICO", "updated_loan_status"], observed=True)
      .size()
      .reset_index(name="count")
)
grouped["total"] = grouped.groupby("avg_FICO", observed=True)["count"].transform("sum")
grouped["percentage"] = grouped["count"] / grouped["total"] * 100

# Plot scatter of percentage breakdown by DTI
fig = px.scatter(
    grouped,
    x="avg_FICO",
    y="percentage",
    color="updated_loan_status",
    symbol='updated_loan_status',
    title="Percentage of Good vs. Bad Loans by Average FICO Score",
    labels={
        "avg_FICO": "Average FICO Score",
        "percentage": "Percentage of Loans (%)",
        "updated_loan_status": "Loan Status"
    },
    color_discrete_map={"Good Loan": "green", "Bad Loan": "red"},
    opacity=0.7
)

fig.update_layout(yaxis=dict(range=[0,100]),
                  legend_title_text='Loan Status')
fig.show()

#### 'updated_loan_status' by 'issue_d'

In [ ]:
# Compute percentage of bad loans by issue date
grouped = (
    cleaned_loan_individual_filled_df.groupby('issue_d')['bad_loan']
      .mean()
      .mul(100)
      .reset_index(name='pct_bad_loans')
)

# Plot line chart
fig = px.line(
    grouped,
    x='issue_d',
    y='pct_bad_loans',
    markers=True,
    title='Percentage of Bad Loans Over Time (by Issue Date)',
    labels={
        'issue_d': 'Issue Date',
        'pct_bad_loans': 'Percentage of Bad Loans (%)'
    }
)

# Retain datetime formatting on the x-axis
fig.update_xaxes(tickformat='%b-%Y')

fig.show()

#### Breakdown of 'updated_loan_status' by 'delinq_2_years'

In [ ]:
# Compute count and percentage breakdown by delinq_2yrs and status
grouped = (
    cleaned_loan_individual_filled_df.groupby(['delinq_2yrs', 'updated_loan_status'], observed=True)
      .size()
      .reset_index(name='count')
)
grouped['total'] = grouped.groupby('delinq_2yrs')['count'].transform('sum')
grouped['percentage'] = grouped['count'] / grouped['total'] * 100

# Sort by delinq_2yrs
grouped = grouped.sort_values('delinq_2yrs')

# Plot interactive line chart
fig = px.line(
    grouped,
    x='delinq_2yrs',
    y='percentage',
    color='updated_loan_status',
    markers=True,
    title='Percentage of Good vs. Bad Loans by Number of Delinquencies (Last 2 Years)',
    labels={
        'delinq_2yrs': 'Delinquencies in Past 2 Years',
        'percentage': 'Percentage of Loans (%)',
        'updated_loan_status': 'Loan Status'
    },
    color_discrete_map={'Good Loan': 'green', 'Bad Loan': 'red'}
)
fig.update_layout(
    xaxis=dict(dtick=1),
    yaxis=dict(range=[0, 100]),
    legend_title_text='Loan Status',
    margin=dict(t=80, b=50)
)
fig.show()

#### Breakdown of 'updated_loan_status' by 'addr_state'

In [ ]:
# Compute % breakdown by state
grp = (
    cleaned_loan_individual_filled_df.groupby(["addr_state", "updated_loan_status"], observed=True)
      .size()
      .reset_index(name="count")
)
grp["total"] = grp.groupby("addr_state", observed=True)["count"].transform("sum")
grp["percent"] = grp["count"] / grp["total"] * 100

# Pivot and sort states by descending Bad‐Loan %
pivot = (
    grp.pivot(index="addr_state", columns="updated_loan_status", values="percent")
       .fillna(0)
       .reset_index()
)
pivot = pivot.sort_values("Bad Loan", ascending=False)

# Melt for plotting
plot_df = pivot.melt(
    id_vars="addr_state",
    value_vars=["Good Loan", "Bad Loan"],
    var_name="Loan Status",
    value_name="Percentage"
)

# Draw grouped bar chart
fig = px.bar(
    plot_df,
    x="addr_state",
    y="Percentage",
    color="Loan Status",
    barmode="group",
    category_orders={"addr_state": pivot["addr_state"].tolist()},
    color_discrete_map={"Good Loan": "green", "Bad Loan": "red"},
    title="Percentage of Good vs. Bad Loans by State (sorted by Bad %)",
    labels={
        "addr_state": "State",
        "Percentage": "Percentage of Loans (%)",
        "Loan Status": "Loan Status",
    },
    text="Percentage"
)
fig.update_traces(texttemplate="%{text:.2f}%", textposition="outside")
fig.update_layout(
    xaxis_tickangle=-45,
    yaxis=dict(title="Percentage of Loans (%)", range=[0, 100]),
    legend_title_text="Loan Status",
    margin=dict(t=100, b=150)
)
fig.show()

#### Breakdown of 'updated_loan_status' by 'region' of the US

In [ ]:
# Compute % by region
grp = (cleaned_loan_individual_filled_df.groupby(['region','updated_loan_status'], observed=True)
          .size()
          .reset_index(name='count'))
grp['total'] = grp.groupby('region', observed=True)['count'].transform('sum')
grp['percentage'] = grp['count']/grp['total']*100

# Plot
fig = px.bar(
  grp, x='region', y='percentage', color='updated_loan_status',
  barmode='group',
  category_orders={'region': ['Northeast','Midwest','South','West']},
  color_discrete_map={'Good Loan':'green','Bad Loan':'red'},
  title='% of Good vs. Bad Loans by Region',
  labels={'percentage':'% of Loans','region':'Region','updated_loan_status':'Loan Status'},
  text='percentage'
)
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(yaxis=dict(range=[0,100]), xaxis_tickangle=-45)
fig.show()

#### Breakdown of 'updated_loan_status' by 'rank_of_state_by_median_household_income_2019'

In [ ]:
# Group by rank and status to compute counts & percentages
grouped = (
    cleaned_loan_individual_filled_df
    .groupby(['rank_of_state_by_median_household_income_2019', 'updated_loan_status'], observed=True)
    .size()
    .reset_index(name='count')
)
grouped['total'] = grouped.groupby('rank_of_state_by_median_household_income_2019', observed=True)['count'].transform('sum')
grouped['percentage'] = grouped['count'] / grouped['total'] * 100

# Sort by rank
grouped = grouped.sort_values('rank_of_state_by_median_household_income_2019')

# Plot breakdown as a line chart
fig = px.line(
    grouped,
    x='rank_of_state_by_median_household_income_2019',
    y='percentage',
    color='updated_loan_status',
    markers=True,
    title='Percentage of Good vs. Bad Loans by State Income Rank (2019)',
    labels={
        'rank_of_state_by_median_household_income_2019': 'State Median Income Rank (1=Highest)',
        'percentage': 'Percentage of Loans (%)',
        'updated_loan_status': 'Loan Status'
    },
    color_discrete_map={'Good Loan': 'green', 'Bad Loan': 'red'}
)
fig.update_layout(yaxis=dict(range=[0, 100]))
fig.show()

#### 'annual_income_more_than_median_household_income_2019' by 'loan status'

In [ ]:
# Compute percentage breakdown by the yes/no flag and loan status
grp = (
    cleaned_loan_individual_filled_df
    .groupby(['annual_income_more_than_median_household_income_2019', 'updated_loan_status'], observed=True)
    .size()
    .reset_index(name='count')
)
grp['total'] = grp.groupby('annual_income_more_than_median_household_income_2019', observed=True)['count'].transform('sum')
grp['percentage'] = grp['count'] / grp['total'] * 100

# 4. Plot grouped bar chart
fig = px.bar(
    grp,
    x='annual_income_more_than_median_household_income_2019',
    y='percentage',
    color='updated_loan_status',
    barmode='group',
    category_orders={'annual_income_more_than_median_household_income_2019': ['yes', 'no']},
    color_discrete_map={'Good Loan': 'green', 'Bad Loan': 'red'},
    title='Loan Status Breakdown by Annual Income > State Median',
    labels={
        'annual_income_more_than_median_household_income_2019': 'Annual Income > Median Household Income?',
        'percentage': 'Percentage of Loans (%)',
        'updated_loan_status': 'Loan Status'
    },
    text='percentage'
)
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(
    yaxis=dict(title='Percentage of Loans (%)', range=[0, 100]),
    xaxis_title='Annual Income > Median Household Income?',
    legend_title='Loan Status',
    margin=dict(t=80, b=50)
)
fig.show()

#### '%_more_or_less' against 'updated_loan_status'

In [ ]:
# 1. Group by % difference and status to compute counts & percentages
grouped = (
    cleaned_loan_individual_filled_df
    .groupby(['%_more_or_less_than_median', 'updated_loan_status'], observed=True)
    .size()
    .reset_index(name='count')
)
grouped['total'] = grouped.groupby('%_more_or_less_than_median', observed=True)['count'].transform('sum')
grouped['percentage'] = grouped['count'] / grouped['total'] * 100

# 2. Plot scatter of percentage vs. % difference
fig = px.scatter(
    grouped,
    x='%_more_or_less_than_median',
    y='percentage',
    color='updated_loan_status',
    symbol='updated_loan_status',
    opacity=0.7,
    title='Percentage of Good vs. Bad Loans by % Difference from Median Income',
    labels={
        '%_more_or_less_than_median': '% Difference from Median Income',
        'percentage': 'Percentage of Loans (%)',
        'updated_loan_status': 'Loan Status'
    },
    color_discrete_map={'Good Loan': 'green', 'Bad Loan': 'red'}
)
fig.update_layout(
    yaxis=dict(range=[0, 100]),
    legend_title_text='Loan Status'
)
fig.show()

### Further Exploratory Data Analysis: Feature selection based on statistical significance

#### Categorical - Continuous Split

In [ ]:
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from scipy import stats

In [ ]:
cleaned_loan_individual_filled_df.info(verbose=True, show_counts=True)

In [ ]:
# subsetting categorical features to run chi-square test
cat_col = cleaned_loan_individual_filled_df.select_dtypes(include=['object'])
print(cat_col.shape)
cat_col.head()

In [ ]:
# subsetting continuous features to run anova/KS test
cont_col = cleaned_loan_individual_filled_df.select_dtypes(include=['int64','float64'])
cont_col.drop(['bad_loan'], axis=1,inplace=True)
print(cont_col.shape)
cont_col.head()

In [ ]:
# joining target variable, updated_loan_status into continuous features dataframe
updated_loan_status = cleaned_loan_individual_filled_df[['updated_loan_status']]
cont_col = cont_col.join(updated_loan_status)
print(cont_col.shape)
cont_col.head()

### Categorical features selection

In [ ]:
# label encoding for categorical features with more than 2 values
label_encoder = LabelEncoder()

cat_col_transformed_dict = {}

for col in cat_col.columns:
    cat_col_transformed = label_encoder.fit_transform(cat_col[col])
    cat_col_transformed_dict[col] = cat_col_transformed
    
cat_col_transformed = pd.DataFrame(cat_col_transformed_dict)
print(cat_col_transformed.shape)
cat_col_transformed.head()

In [ ]:
# chi-square test
X = cat_col_transformed.drop(['updated_loan_status'],axis=1)
y = cat_col_transformed['updated_loan_status']

chi_scores = chi2(X,y)
chi_scores

In [ ]:
# top correlated features + p_values visualization from chi-square test
p_values = pd.Series(chi_scores[1],index = X.columns)
p_values.sort_values(inplace = True)

fig=px.bar(p_values, labels={'value': 'p_value'}, width=600, height=500)
fig.add_hline(y=0.05, line_dash="dash", line_color="red")
fig.update_layout(showlegend=False)
fig.update_xaxes(tickangle = 90)

In [ ]:
# top correlated features + p_values converted to dataframe for top features list extraction
cat_pvals_df = pd.DataFrame(p_values, columns=['p_value'])
cat_pvals_df

In [ ]:
# list of top correlated categorical features
top_cat_features_list = cat_pvals_df[cat_pvals_df['p_value']<0.05].index.to_list()
top_cat_features_list

### Continuous features selection

In [ ]:
# extracting all continuous feature column
cont_col_feature_cols = [col for col in cont_col.columns if col not in ['updated_loan_status']]
print(len(cont_col_feature_cols))
cont_col_feature_cols

In [ ]:
# running normality and statistical significance tests on continuous features
results_dict = {col:cont_stat_sig_test(cont_col,col) for col in cont_col_feature_cols}

cont_stat_results_df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['p_value']).sort_values(by='p_value')

fig=px.bar(cont_stat_results_df, labels={'value': 'p_value'}, width=600, height=500)
fig.add_hline(y=0.05, line_dash="dash", line_color="red")
fig.update_layout(showlegend=False)
fig.update_xaxes(tickangle = 90)

In [ ]:
# continuous features with statistical significance correlation to response
cont_stat_results_df

In [ ]:
# rearranging cont_col dataset's feature columns in the order of statistical significance
cont_col = cont_col[['updated_loan_status']+cont_stat_results_df.index.tolist()]
cont_feature_cols = cont_col.columns.tolist()
cont_feature_cols.remove('updated_loan_status')
print(cont_col.shape)
cont_col.head()

In [ ]:
modelling_loan_individual_filled_df = cleaned_loan_individual_filled_df.copy()

## Dummy Coding

In [ ]:
modelling_loan_individual_filled_df.head()

In [ ]:
modelling_loan_individual_filled_df.info(verbose=True)

### grade

In [ ]:
# Sorting the ordinal features in correct order
grade_to_numeric = {'A': 1, 'B': 1, 'C': 1, 'D': 1,
                    'E': 2, 'F': 3, 'G': 4}

# Create a new column with numeric grades
modelling_loan_individual_filled_df['updated_grade'] = modelling_loan_individual_filled_df['grade'].map(grade_to_numeric)

# Verify the mapping
modelling_loan_individual_filled_df[['grade', 'updated_grade']].head()

In [ ]:
modelling_loan_individual_filled_df['updated_grade'].unique()

### home_ownership

In [ ]:
modelling_loan_individual_filled_df['home_ownership'].unique()

In [ ]:
# Dummy code the 'home_ownership' column

# Create dummy variables for the 'home_ownership' column
home_ownership_dummies = pd.get_dummies(modelling_loan_individual_filled_df.home_ownership, prefix='home_ownership')
home_ownership_dummies.drop(home_ownership_dummies.columns[0], axis=1, inplace=True)

# If you want to add these new dummy columns back into your original DataFrame, use pd.concat:
modelling_loan_individual_filled_df = pd.concat([modelling_loan_individual_filled_df, home_ownership_dummies], axis=1)

### purpose

In [ ]:
# Define the three purposes to keep
dummy_purpose = ['small_business', 'moving', 'renewable_energy']

# Create a grouped‐purpose column
modelling_loan_individual_filled_df['updated_purpose'] = modelling_loan_individual_filled_df['purpose'].where(modelling_loan_individual_filled_df['purpose'].isin(dummy_purpose), other='others')

# One-hot encode that grouping
purpose_dummies = pd.get_dummies(modelling_loan_individual_filled_df.updated_purpose, prefix='purpose')
purpose_dummies.drop(purpose_dummies.columns[0], axis=1, inplace=True)

# Attach the dummies back to your DataFrame
modelling_loan_individual_filled_df = pd.concat([modelling_loan_individual_filled_df, purpose_dummies], axis=1)

### addr_state

In [ ]:
modelling_loan_individual_filled_df['addr_state'].unique()

In [ ]:
# Dummy code the 'addr_state' column

# Create dummy variables for the 'home_ownership' column
addr_state_dummies = pd.get_dummies(modelling_loan_individual_filled_df.addr_state, prefix='addr_state')
addr_state_dummies.drop(addr_state_dummies.columns[0], axis=1, inplace=True)

# If you want to add these new dummy columns back into your original DataFrame, use pd.concat:
modelling_loan_individual_filled_df = pd.concat([modelling_loan_individual_filled_df, addr_state_dummies], axis=1)

In [ ]:
modelling_loan_individual_filled_df.head()

## Modelling

### Feature Columns (X)

In [ ]:
feature_cols = ['loan_amnt','term','int_rate','installment', #'last_pymnt_amnt',
                'updated_grade',# 'grade',# 'sub_grade',
                'emp_length_int',
                'home_ownership_MORTGAGE', 'home_ownership_NONE', 'home_ownership_OTHER', 
                'home_ownership_OWN',	'home_ownership_RENT',# 'home_ownership',
                'annual_inc',# 'cleaned_verification_status',

                'purpose_others', 'purpose_renewable_energy', 'purpose_small_business',# 'purpose', 'updated_purpose'
                
                'dti','delinq_2yrs','avg_FICO',#'avg_last_FICO',
                'inq_last_6mths',
                
                'open_acc','pub_rec',#'total_acc',
                'revol_bal',#'tot_cur_bal','avg_cur_bal',
                'revol_util',#'bc_open_to_buy','bc_util',
                
                # 'out_prncp','out_prncp_inv',
                'total_pymnt',
                # 'total_pymnt_inv',
                # 'total_rec_prncp','total_rec_int',
                'total_rec_late_fee',
                # 'recoveries','collection_recovery_fee',
                
                'collections_12_mths_ex_med', 
                # 'acc_now_delinq',
                'tot_coll_amt',
                
                'acc_open_past_24mths',
                # 'chargeoff_within_12_mths',# 'delinq_amnt',
                'mo_sin_old_il_acct','mo_sin_old_rev_tl_op',#'mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl',
                'mort_acc',#'mths_since_recent_bc',
                
                'num_accts_ever_120_pd','num_actv_bc_tl',#'num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl',
                'num_op_rev_tl',#'num_rev_accts','num_rev_tl_bal_gt_0',
                'num_sats',
                # 'num_tl_120dpd_2m', # 'num_tl_30dpd',
                'num_tl_90g_dpd_24m','num_tl_op_past_12m',
                
                'pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim',
                'total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit',
                                
                # 'hardship_flag',
                # 'debt_settlement_flag',
   
                # 'median_household_income_2019',
                'rank_of_state_by_median_household_income_2019',
                # 'annual_income_more_than_median_household_income_2019',
                '%_more_or_less_than_median'
                # 'addr_state'
                # 'addr_state_AL','addr_state_AR','addr_state_AZ','addr_state_CA','addr_state_CO','addr_state_CT',
                # 'addr_state_DC','addr_state_DE','addr_state_FL','addr_state_GA','addr_state_HI','addr_state_IA', 
                # 'addr_state_ID', 'addr_state_IL', 'addr_state_IN', 'addr_state_KS', 'addr_state_KY', 'addr_state_LA', 
                # 'addr_state_MA', 'addr_state_MD', 'addr_state_ME', 'addr_state_MI', 'addr_state_MN', 'addr_state_MO', 
                # 'addr_state_MS', 'addr_state_MT', 'addr_state_NC', 'addr_state_ND', 'addr_state_NE', 'addr_state_NH', 
                # 'addr_state_NJ', 'addr_state_NM', 'addr_state_NV', 'addr_state_NY', 'addr_state_OH', 'addr_state_OK', 
                # 'addr_state_OR', 'addr_state_PA', 'addr_state_RI', 'addr_state_SC', 'addr_state_SD', 'addr_state_TN', 
                # 'addr_state_TX', 'addr_state_UT', 'addr_state_VA', 'addr_state_VT', 'addr_state_WA', 'addr_state_WI', 
                # 'addr_state_WV', 'addr_state_WY' 
               ]

X = modelling_loan_individual_filled_df[feature_cols]

### Target Variable (y)

In [ ]:
y = modelling_loan_individual_filled_df['bad_loan']

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split, PredefinedSplit
from sklearn.metrics import make_scorer, accuracy_score, recall_score, f1_score, precision_score, confusion_matrix, classification_report, precision_recall_curve

# Split into train/test, preserving the imbalance via stratify

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,         # 20% for testing; adjust as needed
    random_state=42,
    stratify=y             # keeps the percentage of target variable consistent in both sets
)

### Modelling using Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline # load a tool that lets us string together several steps into one simple process
from sklearn.preprocessing import StandardScaler  # load a tool that makes each column of numbers roughly the same size, so that they are balanced
from sklearn.model_selection import GridSearchCV, StratifiedKFold  # load tools to try many different settings and to split our data in a way that keeps the same mix of good vs. bad loans each time

#### Step 1: Training Base Logistic Regression Model without Hyperparameter Tuning

In [ ]:
# Initialize logistic regression with balanced class weights
base_log_reg_model = LogisticRegression(
    class_weight='balanced',  # weight the minority class more
    solver='liblinear',       # good default for small-to-medium datasets
    max_iter=1000,
    random_state=42
)

In [ ]:
# Train the model
base_log_reg_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = base_log_reg_model.predict(X_test)

# Evaluate
print(f'Recall: {recall_score(y_test, y_pred)}')
print('\nConfusion Matrix:')
print(confusion_matrix(y_test, y_pred))
print('\nClassification Report:')
print(classification_report(y_test, y_pred))

#### Step 2a: Hyperparameter Tuning for base_log_reg_model, optimising for Precision, whilst keeping runtime <30 minutes

In [ ]:
# 1) Split off 20% of your training data for validation, keeping the same 12.5% bad-loan rate:
X_subtrain, X_val, y_subtrain, y_val = train_test_split(
    X_train,
    y_train,
    test_size=0.20,
    stratify=y_train,
    random_state=42
)

# 2) Tell sklearn which rows are “train” (–1) vs “validation” (0):
test_fold = np.concatenate([
    np.full(len(X_subtrain), -1, dtype=int),
    np.zeros(len(X_val),       dtype=int)
])
ps = PredefinedSplit(test_fold)

# 3) Our “scale → classify” recipe, with a lighter iteration cap:
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(
        class_weight='balanced',
        max_iter=300,             
        random_state=42
    ))
])

# 4) A much smaller knob-set to spin:
param_distributions = [
    {
        "clf__solver": ["liblinear"],           
        "clf__penalty": ["l1", "l2"], 
        "clf__C": [0.1, 1, 10]                   
    }
]

# 5) Set up a much shorter randomized search:
search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_distributions, 
    n_iter=6,                                  
    scoring="precision",
    cv=ps,
    n_jobs=1,
    verbose=2,
    random_state=42
)

# 6) Combine and run:
X_combined = np.vstack([X_subtrain, X_val])
y_combined = np.concatenate([y_subtrain, y_val])
search.fit(X_combined, y_combined)

# 7) Report validation‐fold precision:
print("Best settings:", search.best_params_)
print("Best precision (validation):", f"{search.best_score_:.3f}")

# 8) Final test‐set check:
best_log_reg_model = search.best_estimator_
y_pred = best_log_reg_model.predict(X_test)

print(f"\nTest accuracy : {accuracy_score(y_test, y_pred):.4f}")
print(f"Test precision: {precision_score(y_test, y_pred):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


#### Step 3: Improve Precision Scores (i.e. Precision ≥ 0.7) using Threshold Calibration on 'base_log_reg_model'

In [ ]:
# 1) Get the predicted probability for the positive class on your test set
y_probability = best_log_reg_model.predict_proba(X_test)[:, 1]

# 2) Compute the precision-recall curve
precisions, recalls, thresholds = precision_recall_curve(y_test, y_probability)

# 3) Find all thresholds that give precision ≥ 0.7
# Note: `precisions[i+1]` corresponds to `thresholds[i]`
valid_idxs = np.where(precisions[1:] >= 0.7)[0]

if len(valid_idxs) == 0:
    raise ValueError("No threshold achieves precision ≥ 0.7")

# 4) Pick the threshold that maximizes recall among those
best_idx = valid_idxs[np.argmax(recalls[1:][valid_idxs])]
best_threshold = thresholds[best_idx]
print(f"Chosen probability threshold for precision ≥ 0.7: {best_threshold:.3f}")

# 5) Apply that threshold to get final predictions
y_pred_threshold = (y_probability >= best_threshold).astype(int)

# 6) Evaluate
print(f"Accuracy:  {accuracy_score(y_test, y_pred_threshold):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_threshold):.4f}")
print(f"Recall:    {recall_score(y_test, y_pred_threshold):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_threshold))
print("\nClassification Report:")
print(classification_report(y_test, y_pred_threshold))

### Modelling using XGBoost

In [ ]:
import xgboost
from xgboost import XGBClassifier

#### Step 1: Training Base XGBoost Model without Hyperparameter Tuning

In [ ]:
# Compute scale_pos_weight to balance the classes
# Counts how many examples in your training labels are “negative” (class 0) and how many are “positive” (class 1).
neg, pos = np.bincount(y_train)

In [ ]:
# Instantiate the classifier
base_xgb_model = XGBClassifier( 
    objective='binary:logistic', # Learning Task Parameter: for binary classification
    scale_pos_weight=neg / pos, # Booster Parameter: Uses the ratio of neg/pos to ensure that the values are weighted
    eval_metric='logloss', #  Learning Task Parameter: Chooses the “log loss” score for measuring mistakes during training. usually selected for binary:logistic
    random_state=42
)

In [ ]:
# Training the Model
base_xgb_model.fit(X_train, y_train)

# Predicting on test set
y_pred = base_xgb_model.predict(X_test)
y_pred_proba = base_xgb_model.predict_proba(X_test)[:, 1]  # Instead of just 0/1, provides confidence on the probability the model assigns to class 1 for each test example.

# Compute & print metrics
print(f"Recall: {recall_score(y_test, y_pred):.4f}\n")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred), "\n")
print("Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

#### Step 2: A Randomized Hyperparameter Search on xgb_model, optimising for Precision

In [ ]:
# 1) Split into subtrain / validation
X_sub, X_val, y_sub, y_val = train_test_split(
    X_train, y_train,
    test_size=0.2,
    stratify=y_train,
    random_state=42
)

# 2) Further downsample sub-training set to 50 K rows for hyperparameter search 
X_tune, _, y_tune, _ = train_test_split( #_, _: The underscores mean “we don’t care about the leftover rows right now.”
    X_sub, y_sub,
    train_size=50000,
    stratify=y_sub,
    random_state=42
)

# 3) Base XGB with imbalance handled
hyperparameter_xgb_model = XGBClassifier(
    eval_metric='logloss',
    scale_pos_weight=(y_train==0).sum() / (y_train==1).sum(),
    random_state=42
)

# 4) Precision scorer
precision_scorer = make_scorer(precision_score) #Wraps scikit-learn’s precision_score into a form that Grid/Random search can use.

# 5) Choosing the hyperparameters
param_dist = {
    'n_estimators':     [100, 150, 200], # try building forests of 100, 150, or 200 trees
    'max_depth':        [3, 5, 7], # allow each tree be shallow (depth 3), medium (5) or a bit deeper (7)
    'learning_rate':    [0.01, 0.05, 0.1], # try slow or faster learning rates
    'subsample':        [0.8, 1.0], # whether to sample 80% or 100% of rows per tree
    'colsample_bytree': [0.8, 1.0] # whether to sample 80% or 100% of columns per tree
}

# 6) Tuning our hyperparameter_xgb_model thru randomised search (20 samples, 3-fold CV)
rand_search = RandomizedSearchCV(
    estimator=hyperparameter_xgb_model,
    param_distributions=param_dist,
    n_iter=20,                # Only try 20 random combinations out of the full menu (saves time vs. brute-forcing every combo)
    scoring=precision_scorer, # Rank combos by precision
    cv=3,                     # For each combo, do a quick 3-way “train/test” split to see how it generalizes.
    n_jobs=-1, # Use all CPU cores you can
    random_state=42,
    verbose=1
)

# 7) Fit on the smaller tuning set
rand_search.fit(X_tune, y_tune) # Goes through 20 random model settings, does 3-fold CV each time, and picks the one with highest precision.
print("Best precision params:", rand_search.best_params_) # Tells you which combination "won"

# 8) Training the winning model on the full sub-train set
best_xgb_model = rand_search.best_estimator_ # Uses the "champion" model (with its chosen hyperparameters)
best_xgb_model.set_params(**rand_search.best_params_)
best_xgb_model.fit( # Teaches it on all 80% of the data we kept aside in step 1
    X_sub,
    y_sub,
    verbose=False
)

# 9) Final evaluation on the "untouched" test set
y_pred = best_xgb_model.predict(X_test)

print(f"Recall: {recall_score(y_test, y_pred):.4f}\n")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred), "\n")
print("Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

#### Step 3a: Improve Precision Scores (i.e. Precision ≥ 0.7) using Threshold Calibration on 'best_xgb_model'

In [ ]:
# Assume 'best_xgb_model' is the final fitted model and X_test, y_test are ready.

# Ask the model: “For each test example, what’s the chance it belongs to class 1 (the ‘positive’ class)?”
# Predict_proba gives you two numbers per example (chance of 0 vs. chance of 1) and [:,1] picks out the “chance of 1.”
y_probability = best_xgb_model.predict_proba(X_test)[:,1] 

# Building the precision/recall trade‐off: comparing the y_test to those predicted probabilities (probs).
# Sweep through many possible cut‐off points (i.e. “thresholds”) — from “call it 1 if prob≥0.0” up to “prob≥1.0” — and at each cut-off we record how precise we’d be (of everything we call 1, how many really are 1) and how complete we’d be (of all the actual 1s, how many we catch).
# That gives us three lists of numbers: the resulting precision, recall, and the thresholds that produced them.
precision, recall, thresholds = precision_recall_curve(y_test, y_probability)

# Identify smallest threshold that gives precision ≥ 0.70
idx = np.where(precision >= 0.70)[0] # Look through our precision list and find every position where precision is at least 70%. np.where(...) returns all the spots that meet the rule. [0] just pulls out the raw list of those positions.
if len(idx): # “Do we have any thresholds at all that yield ≥70% precision?” If our list of good spots (idx) isn’t empty, go inside the if; otherwise skip to the else.
    thr = thresholds[idx[0]] # Pick the very first (smallest) threshold from that list—i.e. the easiest cut-off that already gets us to 70% precision.
    print(f"Use threshold = {thr:.4f} to get precision ≥ 0.7")
    y_pred = (y_probability >= thr).astype(int) # Classify using that new cut-off: If the model’s chance ≥ our chosen threshold, call it 1; otherwise call it 0. .astype(int) just turns those True/False answers into 1/0. 
    print(f"Recall: {recall_score(y_test, y_pred):.4f}\n")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred), "\n")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, digits=4))    
else:
    print("Cannot reach precision ≥ 0.7 by thresholding alone.")

#### Step 3b: Improve F₁ Scores (i.e. F₁ ≥ 0.7) using Threshold Calibration on 'best_xgb_model'

In [ ]:
# 1) Get your model’s positive-class probabilities
y_probability = best_xgb_model.predict_proba(X_test)[:, 1]

# 2) Compute precision, recall and thresholds
precision, recall, thresholds = precision_recall_curve(y_test, y_probability)

# 3) Compute F1 at each threshold
f1_scores = 2 * precision * recall / (precision + recall + 1e-8)

# 4) Option I: pick the threshold that maximises F1
best_idx = np.argmax(f1_scores)
best_thr = thresholds[best_idx]
print(f"Max F1 = {f1_scores[best_idx]:.4f} at threshold = {best_thr:.4f}")

# 5) Option II: if you *must* hit ≥0.7, find the first threshold that does
idxs = np.where(f1_scores >= 0.7)[0]
if len(idxs):
    thr_07 = thresholds[idxs[0]]
    print(f"Threshold {thr_07:.4f} yields F1 = {f1_scores[idxs[0]]:.4f}")
    use_thr = thr_07
else:
    print("No threshold yields F1 ≥ 0.7; using max‐F1 threshold instead.")
    use_thr = best_thr

# 6) Apply threshold for Option II (i.e. F₁ = 0.7)
y_pred_threshold = (y_probability >= use_thr).astype(int)

# 7) Print full classification report

print(f"Recall: {recall_score(y_test, y_pred_threshold):.4f}\n")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_threshold), "\n")
print("Classification Report:")
print(classification_report(y_test, y_pred_threshold, digits=4))   

#### Plotting the Precision-Recall Curve for 'best_xgb_model'

In [ ]:
# Drop the last recall/precision (they pair with no threshold)
recall_pts = recall[:-1]
precision_pts = precision[:-1]
thresholds_pts = thresholds

# Make an interactive line+marker plot, with threshold on hover
fig = px.line(
    x=recall_pts,
    y=precision_pts,
    markers=True,
    labels={'x':'Recall', 'y':'Precision'},
    hover_data={'threshold': thresholds_pts}
)
fig.update_layout(title='Precision-Recall Curve for best_xgb_model')
fig.show()


#### Determine Information Value (IV) & Weight of Evidence (WOE)

In [ ]:
def calc_woe_iv(df, feature, target):
    """
    Calculate Weight of Evidence (WOE) and Information Value (IV) for one feature.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Your dataset.
    feature : str
        The name of the feature/column to analyze.
    target : str
        The name of the binary target column (0/1).
    
    Returns
    -------
    iv : float
        Information Value for the feature.
    woe_df : pandas.DataFrame
        Per-bin table with counts, distributions, WOE, and IV contribution.
    """
    # 1) Build the bin table
    lst = []
    totals = df.groupby(feature)[target].count()
    events = df.groupby(feature)[target].sum()
    non_events = totals - events
    total_events = events.sum()
    total_non_events = non_events.sum()
    
    for val in totals.index:
        evt = events.loc[val]
        non_evt = non_events.loc[val]
        dist_evt = evt / total_events
        dist_non_evt = non_evt / total_non_events
        
        # avoid division by zero / log issues
        woe = np.log((dist_evt + 1e-8) / (dist_non_evt + 1e-8))
        iv_bin = (dist_evt - dist_non_evt) * woe
        
        lst.append({
            feature: val,
            'count': totals.loc[val],
            'events': evt,
            'non_events': non_evt,
            'dist_events': dist_evt,
            'dist_non_events': dist_non_evt,
            'WOE': woe,
            'IV_contribution': iv_bin
        })
    
    woe_df = pd.DataFrame(lst).sort_values('WOE')
    iv = woe_df['IV_contribution'].sum()
    
    return iv, woe_df

# Example: calculate for one categorical feature
iv_home_ownership, woe_home_ownership = calc_woe_iv(
    modelling_loan_individual_filled_df, 
    feature='home_ownership', 
    target='bad_loan'
)
print(f"IV for home_ownership: {iv_home_ownership:.4f}")
display(woe_home_ownership)

# ----------------------------------------------------
# If you want IV for *all* categorical features:
# cat_feats = ['home_ownership', 'verification_status', 'purpose', ...]  # fill in your list

# iv_dict = {}
# for feat in cat_feats:
#     iv, _ = calc_woe_iv(lending_club_loan_clean_individual, feat, 'default')
#     iv_dict[feat] = iv

# # Sort features by IV
# iv_series = pd.Series(iv_dict).sort_values(ascending=False)
# print("Features ranked by IV:")
# print(iv_series)

## Exporting to .csv for Dashboard

In [ ]:
dashboard_loan_individual_filled_df = modelling_loan_individual_filled_df.copy()

In [ ]:
dashboard_loan_individual_filled_df.shape

### Dropping Columns that I will not be using in PowerBI

In [ ]:
cols_to_drop_for_dashboard = ['out_prncp','out_prncp_inv','total_pymnt_inv', #'total_pymnt',
                'total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_amnt',
                'collections_12_mths_ex_med', 
                'acc_now_delinq',
                'tot_coll_amt','tot_cur_bal','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy','bc_util',
                'chargeoff_within_12_mths', 'delinq_amnt',
                'mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc',
                'num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl',
                'num_rev_accts','num_rev_tl_bal_gt_0','num_sats',
                'num_tl_120dpd_2m', 'num_tl_30dpd',
                'num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75','pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim',
                'total_bal_ex_mort','total_bc_limit','total_il_high_credit_limit',

                'updated_grade',

                'purpose_others',	'purpose_renewable_energy',	'purpose_small_business',
                'home_ownership_MORTGAGE', 'home_ownership_NONE', 'home_ownership_OTHER', 'home_ownership_OWN',	'home_ownership_RENT',

                'addr_state_AL','addr_state_AR','addr_state_AZ','addr_state_CA','addr_state_CO','addr_state_CT',
                'addr_state_DC','addr_state_DE','addr_state_FL','addr_state_GA','addr_state_HI','addr_state_IA', 
                'addr_state_ID', 'addr_state_IL', 'addr_state_IN', 'addr_state_KS', 'addr_state_KY', 'addr_state_LA', 
                'addr_state_MA', 'addr_state_MD', 'addr_state_ME', 'addr_state_MI', 'addr_state_MN', 'addr_state_MO', 
                'addr_state_MS', 'addr_state_MT', 'addr_state_NC', 'addr_state_ND', 'addr_state_NE', 'addr_state_NH', 
                'addr_state_NJ', 'addr_state_NM', 'addr_state_NV', 'addr_state_NY', 'addr_state_OH', 'addr_state_OK', 
                'addr_state_OR', 'addr_state_PA', 'addr_state_RI', 'addr_state_SC', 'addr_state_SD', 'addr_state_TN', 
                'addr_state_TX', 'addr_state_UT', 'addr_state_VA', 'addr_state_VT', 'addr_state_WA', 'addr_state_WI', 
                'addr_state_WV', 'addr_state_WY'
                  ]

dashboard_loan_individual_filled_df = dashboard_loan_individual_filled_df.drop(cols_to_drop_for_dashboard, axis=1)

In [ ]:
dashboard_loan_individual_filled_df.shape 

In [ ]:
dashboard_loan_individual_filled_df.head()

### Downsampling my Dataset for ease of Import into PowerBI

In [ ]:
# Determining sample size
sample_size = 0.30

loan_sampled, loan_remainder = train_test_split(
    dashboard_loan_individual_filled_df,
    train_size=sample_size,
    stratify=dashboard_loan_individual_filled_df['bad_loan'],
    random_state=42
)

# Check on whether ratio in loan_sampled = dashboard_loan_individual_filled_df
print("Original ratios:\n", dashboard_loan_individual_filled_df['bad_loan'].value_counts(normalize=True))
print("Sampled ratios:\n", loan_sampled['bad_loan'].value_counts(normalize=True))

### Exporting the sample dataset

In [ ]:
loan_sampled.to_csv('./dashboard_loan_individual_filled_df.csv', index=False)